In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm


def convert_time_interval(input_interval):
    """
    Function to convert a time interval in hours, days, or minutes to minutes.

    Parameters:
        input_interval (str): The time interval as a string.
                              For example: '5H' for 5 hours, '3D' for 3 days, or '300T' for 300 minutes.

    Returns:
        int: The equivalent time interval in minutes.
    """
    num = int(input_interval[:-1])  # Get the number (ignore the last character 'H', 'D', or 'T').
    unit = input_interval[-1]       # Get the unit of time ('H' for hours, 'D' for days, or 'T' for minutes).

    if unit == 'H':
        return f"{int(num/10 * 60)}T", f"{num * 60}T"  # Convert hours to minutes.
    elif unit == 'D':
        return f"{int(num/10 * 24 * 60)}T", f"{num * 24 * 60}T"  # Convert days to minutes.
    elif unit == 'T':
        if num<=5:
            return f"1T", f"{num}T"  # For minutes, return the value directly.
        else:
            return f"{int(num/5)}T", f"{num}T"  # For minutes, return the value directly.
    else:
        raise ValueError("Invalid input interval. Please use 'H' for hours, 'D' for days, or 'T' for minutes.")


def resample_data(df, granularity):
    resample = df.resample(granularity)

    df_resampled = pd.DataFrame()
    df_resampled["open"] = resample["open"].first()
    df_resampled["high"] = resample["high"].max()
    df_resampled["low"] = resample["low"].min()
    df_resampled["close"] = resample["close"].last()
    df_resampled["tick_volume"] = resample["tick_volume"].sum()

    return df_resampled


def find_timestamp_extremum(df, df_lower_timeframe):
    """
    :param: df_lowest_timeframe
    :return: self._data with three new columns: Low_time (TimeStamp), High_time (TimeStamp), High_first (Boolean)
    """

    df = df.copy()
    df = df.loc[df_lower_timeframe.index[0]:]

    # Set new columns
    df["low_time"] = np.nan
    df["high_time"] = np.nan

    # Loop to find out which of the high or low appears first
    for i in tqdm(range(len(df) - 1)):

        # Extract values from the lowest timeframe dataframe
        start = df.iloc[i:i + 1].index[0]
        end = df.iloc[i + 1:i + 2].index[0]
        row_lowest_timeframe = df_lower_timeframe.loc[start:end].iloc[:-1]

        # Extract Timestamp of the max and min over the period (highest timeframe)
        try:
            high = row_lowest_timeframe["high"].idxmax()
            low = row_lowest_timeframe["low"].idxmin()

            df.loc[start, "low_time"] = low
            df.loc[start, "high_time"] = high

        except Exception as e:
            print(e)
            df.loc[start, "low_time"] = None
            df.loc[start, "high_time"] = None

    # Verify the number of row without both TP and SL on same time
    percentage_good_row = len(df.dropna()) / len(df) * 100

    # if percentage_garbage_row<95:
    print(f"Good rows: {'%.2f' % percentage_good_row} %")

    df = df.iloc[:-1]

    return df

In [14]:
symbols = ["EURUSD","GBPUSD", "USDCHF", "USDJPY", "USDCAD", "BTCEUR", "BTCGBP", "BTCUSD", "BTCAUD", "ETHAUD", "LTCUSD",
           "ETHEUR", "ETHGBP", "ETHUSD", "US30", "UK100", "NAS100", "US500", "JPN225", "AUS200", "FRA40"]

new_tf = input("Timeframe (without quotes):")

for symbol in symbols:
    path = f"1-minute Pepperstone/1-minute{symbol}.csv"
    low_tf, high_tf = convert_time_interval(new_tf)
    #print(low_tf, high_tf)

    print(" ")
    print("-----------------------------------------------------------------------")
    print("IMPORT 1-MINUTE DATA")

    df_1m = pd.read_csv(path, index_col="time", parse_dates=True)

    print(" ")
    print("-----------------------------------------------------------------------")
    print("START RESAMPLING")


    df_lower = resample_data(df_1m, low_tf).dropna()
    df_higher = resample_data(df_1m, high_tf).dropna()


    df = find_timestamp_extremum(df_higher, df_lower)
    print(symbol)
    print(df)

    path_out = f"4-hours Pepperstone/4-hours_{symbol}.csv"

    if len(path_out) == 0:
        pass
    else:
        df.to_csv(path_out)

Timeframe (without quotes):4H
 
-----------------------------------------------------------------------
IMPORT 1-MINUTE DATA
 
-----------------------------------------------------------------------
START RESAMPLING


100%|███████████████████████████████████| 13676/13676 [00:08<00:00, 1611.73it/s]


Good rows: 99.99 %
EURUSD
                        open     high      low    close  tick_volume  \
time                                                                   
2015-01-02 12:00:00  1.20467  1.20600  1.20347  1.20377        14827   
2015-01-02 16:00:00  1.20377  1.20437  1.20069  1.20131        19160   
2015-01-02 20:00:00  1.20133  1.20154  1.20011  1.20029         9335   
2015-01-03 00:00:00  1.20030  1.20055  1.19997  1.20014         1351   
2015-01-05 00:00:00  1.19454  1.19755  1.18642  1.19624        13941   
...                      ...      ...      ...      ...          ...   
2023-09-19 16:00:00  1.06997  1.07014  1.06785  1.06848        13607   
2023-09-19 20:00:00  1.06848  1.06918  1.06761  1.06790         6544   
2023-09-20 00:00:00  1.06790  1.06860  1.06768  1.06849         2478   
2023-09-20 04:00:00  1.06849  1.06897  1.06783  1.06786         4333   
2023-09-20 08:00:00  1.06786  1.06972  1.06730  1.06964        11014   

                                low_t

100%|███████████████████████████████████| 13676/13676 [00:08<00:00, 1668.74it/s]


Good rows: 99.99 %
GBPUSD
                        open     high      low    close  tick_volume  \
time                                                                   
2015-01-02 12:00:00  1.55296  1.55366  1.54120  1.54255        21790   
2015-01-02 16:00:00  1.54254  1.54386  1.53551  1.53597        22980   
2015-01-02 20:00:00  1.53597  1.53626  1.53268  1.53330        13946   
2015-01-03 00:00:00  1.53328  1.53338  1.53242  1.53244         2171   
2015-01-05 00:00:00  1.52995  1.53184  1.51718  1.53086        16653   
...                      ...      ...      ...      ...          ...   
2023-09-19 16:00:00  1.24055  1.24067  1.23823  1.23869        20849   
2023-09-19 20:00:00  1.23870  1.23966  1.23858  1.23912        10200   
2023-09-20 00:00:00  1.23846  1.23968  1.23756  1.23898         3513   
2023-09-20 04:00:00  1.23899  1.23937  1.23824  1.23884         7389   
2023-09-20 08:00:00  1.23883  1.23954  1.23329  1.23685        18396   

                                low_t

...

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence


 54%|███████████████████▉                 | 3167/5864 [00:01<00:01, 1807.02it/s]

attempt to get argmax of an empty sequence


100%|█████████████████████████████████████| 5864/5864 [00:03<00:00, 1839.48it/s]

Good rows: 99.73 %
FRA40
                       open    high     low   close  tick_volume  \
time                                                               
2018-01-01 16:00:00  5300.0  5300.0  5300.0  5300.0            1   
2018-01-01 20:00:00  5300.0  5300.0  5300.0  5300.0            1   
2018-01-02 08:00:00  5305.5  5328.7  5280.8  5288.8         3306   
2018-01-02 12:00:00  5288.5  5289.5  5255.7  5276.7         4218   
2018-01-02 16:00:00  5277.0  5295.2  5266.2  5281.0         4517   
...                     ...     ...     ...     ...          ...   
2023-09-19 12:00:00  7292.0  7305.1  7268.6  7272.0         9320   
2023-09-19 16:00:00  7272.2  7289.9  7255.7  7275.8        12279   
2023-09-19 20:00:00  7275.5  7305.0  7275.3  7295.0         3300   
2023-09-20 08:00:00  7273.5  7311.8  7267.0  7306.6        11934   
2023-09-20 12:00:00  7306.9  7325.1  7291.2  7322.1         8812   

                                low_time            high_time  
time                      